In [1]:
import pandas as pd
import acquire
import wrangle
# writing original gzip csv to dataframe
# df = pd.read_csv('anonymized-curriculum-access.txt.gz', compression='gzip', header=None, sep=' ', quotechar='"', error_bad_lines=False)

In [2]:
df = acquire.acquire_curr_logs()
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2.0


In [3]:
df.shape

(900223, 15)

In [4]:
df = wrangle.wrangle_curr_logs(df)
df

,path,user,ip,cohort,program,year,month,day,hour
datetime,,,,,,,,,
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,web_dev,2018,1,26,9
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,web_dev,2018,1,26,9
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,web_dev,2018,1,26,9
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,web_dev,2018,1,26,9
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,staff,2018,1,26,9
...,...,...,...,...,...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,71.150.217.33,Staff,staff,2021,4,21,16
2021-04-21 16:42:02,jquery/mapbox-api,64,71.150.217.33,Staff,staff,2021,4,21,16
2021-04-21 16:42:09,jquery/ajax/weather-map,64,71.150.217.33,Staff,staff,2021,4,21,16


### <font color = 'red'>What is?

- User 152, Apollo cohort, program = other; only five appearances (all on 3/22/2018)

In [ ]:
df.value_counts("ip")


# Takeaways 
- Justin and I use iloc to look up our ip addresses and noticed we were both in group 133, but had different numbers.
- We assigned col 4 to "cohort"
- We assigned col 3 to ""

In [5]:
me = df.loc[df.ip == '162.222.247.26']

In [6]:
me

,path,user,ip,cohort,program,year,month,day,hour
datetime,,,,,,,,,
2021-02-22 11:12:17,/,840,162.222.247.26,Easley,data_sci,2021,2,22,11
2021-02-22 11:12:35,classification/overview,840,162.222.247.26,Easley,data_sci,2021,2,22,11
2021-02-22 11:12:35,classification/scale_features_or_not.svg,840,162.222.247.26,Easley,data_sci,2021,2,22,11
2021-02-22 11:12:35,classification/classical_programming_vs_machin...,840,162.222.247.26,Easley,data_sci,2021,2,22,11
2021-02-22 12:06:03,classification/acquire,840,162.222.247.26,Easley,data_sci,2021,2,22,12
...,...,...,...,...,...,...,...,...,...
2021-04-20 20:02:20,/,840,162.222.247.26,Easley,data_sci,2021,4,20,20
2021-04-21 09:11:27,/,840,162.222.247.26,Easley,data_sci,2021,4,21,9
2021-04-21 09:11:36,anomaly-detection/overview,840,162.222.247.26,Easley,data_sci,2021,4,21,9


In [10]:
me.loc[me.hour >= 22]

,path,user,ip,cohort,program,year,month,day,hour
datetime,,,,,,,,,
2021-03-07 22:31:42,/,840,162.222.247.26,Easley,data_sci,2021,3,7,22
2021-03-07 22:32:32,classification/prep,840,162.222.247.26,Easley,data_sci,2021,3,7,22
2021-03-08 23:37:37,/,840,162.222.247.26,Easley,data_sci,2021,3,8,23
2021-03-10 22:36:53,/,840,162.222.247.26,Easley,data_sci,2021,3,10,22
2021-03-10 22:37:10,regression/overview,840,162.222.247.26,Easley,data_sci,2021,3,10,22
2021-03-10 22:37:13,timeseries/acquire,840,162.222.247.26,Easley,data_sci,2021,3,10,22
2021-03-10 22:38:28,regression/acquire-and-prep,840,162.222.247.26,Easley,data_sci,2021,3,10,22
2021-03-10 22:38:59,regression/split-and-scale,840,162.222.247.26,Easley,data_sci,2021,3,10,22
2021-03-10 22:39:38,regression/project,840,162.222.247.26,Easley,data_sci,2021,3,10,22


In [11]:
me.loc[me.hour <= 5]

,path,user,ip,cohort,program,year,month,day,hour
datetime,,,,,,,,,
2021-03-09 00:18:44,/,840,162.222.247.26,Easley,data_sci,2021,3,9,0
2021-03-09 00:18:54,fundamentals/functions,840,162.222.247.26,Easley,data_sci,2021,3,9,0
2021-03-09 00:19:39,classification/random-forests,840,162.222.247.26,Easley,data_sci,2021,3,9,0
2021-04-06 00:04:15,regression/model,840,162.222.247.26,Easley,data_sci,2021,4,6,0
2021-04-11 00:03:46,individual-project/individual-project,840,162.222.247.26,Easley,data_sci,2021,4,11,0
2021-04-11 04:42:28,individual-project/individual-project,840,162.222.247.26,Easley,data_sci,2021,4,11,4
2021-04-11 05:33:16,classification/decision-trees,840,162.222.247.26,Easley,data_sci,2021,4,11,5
2021-04-11 05:37:19,classification/explore,840,162.222.247.26,Easley,data_sci,2021,4,11,5
2021-04-11 05:37:22,search/search_index.json,840,162.222.247.26,Easley,data_sci,2021,4,11,5


In [ ]:
df.loc[df.user == 840]

In [ ]:
#split time columns
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['hour'] = df.index.hour


In [ ]:
df

In [ ]:
# create dataframe out of value counts
ip_counts = pd.DataFrame(df.ip.value_counts()).reset_index()

ip_counts

In [ ]:
# rename columns
ip_counts.columns=['ip', 'event_count']
ip_counts

In [ ]:
# get the number of ip addresses seen 1, or 2, or 11,998 times. 
ip_counts.groupby(['event_count']).count()

In [ ]:
once = ip_counts.loc[ip_counts.event_count == 1]
twice = ip_counts.loc[ip_counts.event_count == 2]
many = ip_counts.loc[ip_counts.event_count >= 15000]
many


In [ ]:
# Takeaways 
- 822 ips seen only once
- 554 seen only twice
- >= 15000 - only two  (97.105.19.61 and 97.105.19.58) both owned by codeup